<a href="https://colab.research.google.com/github/Faouzi-Slimi/Project/blob/main/Fathia_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 199 kB 37.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a127277d18804faa3c06b46c7ed334c775819b4a05981df7765ad2b8001503a8
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [37]:
# read data
import pandas as pd
import numpy as np

data = pd.read_json('/content/drive/MyDrive/jobs.json', orient='records') # Import Data

Data = data['title'] # We will work only with the "titlte" column

# Drop Na Values
data = []
for x in range(len(Data)):
  if len(Data[x] ) > 0 :
     data.append(Data[x][0])

data = pd.DataFrame(data,columns=["text",])
data[:5]






,text
0,Financier femme
1,Développeur back-end
2,Développeur Angular (SDW030)
3,Développeur Android
4,Développeur sénior full stack JS


In [38]:

# Create a spark contex and read the data

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [39]:
# Convert data to dataframe

data = spark.createDataFrame(data)


In [42]:
#### Create Pipeline
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer,HashingTF,IDF
from pyspark.ml.clustering import KMeans
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashigTF = HashingTF(inputCol="words", outputCol="tf", numFeatures=200)
idf = IDF(inputCol="tf",outputCol="idf")
kmeans = KMeans(k=2,featuresCol="idf")

pipeline = Pipeline(stages=[tokenizer,hashigTF,idf,kmeans])



In [43]:

# fit the model

model = pipeline.fit(data)

Exception ignored in: <function JavaWrapper.__del__ at 0x7fcb5bcfa170>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pyspark/ml/wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'KMeans' object has no attribute '_java_obj'


In [44]:
# Transform model

job_clusturing = model.transform(data)

In [53]:
# Select 
from pyspark.sql.functions import col
job_clusturing.select(col("text"),col("prediction")).show()


+--------------------+----------+
|                text|prediction|
+--------------------+----------+
|     Financier femme|         0|
|Développeur back-end|         0|
|Développeur Angul...|         0|
| Développeur Android|         0|
|Développeur sénio...|         1|
|     Développeur IOS|         0|
|technicien inform...|         0|
|Innoyadev recrute...|         0|
|Excellia Créance ...|         0|
|KO’Date offre Sta...|         0|
|Think Tank is loo...|         0|
|Intercargo recrut...|         0|
|Focus Corporation...|         0|
|KO’date offre PFE...|         0|
|technicien inform...|         0|
|Culturel recrute ...|         0|
|Sayyar Technique ...|         0|
|METREUR VERIFICAT...|         0|
|Wall Street Engli...|         0|
|Téléopérateurs(tr...|         0|
+--------------------+----------+
only showing top 20 rows

